# MINI-PROJ-01

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
X_train = X_train_full / 255.
y_train = y_train_full.astype(float)

In [ ]:
tf.__version__

'2.8.2'

In [ ]:
X_train

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [ ]:
y_train

array([9., 0., 0., ..., 3., 0., 5.])

In [ ]:
print(X_train.shape, y_train.shape)

(60000, 28, 28) (60000,)


In [ ]:
print(X_test.shape)

(10000, 28, 28)


In [ ]:
def build_model():
    from functools import partial

    DefaultConv2D = partial(keras.layers.Conv2D,
                            kernel_size=2, kernel_initializer = 'lecun_normal',
                            activation='relu', padding="SAME")

    model = keras.models.Sequential([
        DefaultConv2D(filters=64, kernel_size=3, input_shape=[28, 28,1]),
        keras.layers.MaxPooling2D(pool_size=2),
        DefaultConv2D(filters=128),
        keras.layers.BatchNormalization(),
        DefaultConv2D(filters=128),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(pool_size=2),
        DefaultConv2D(filters=256),
        keras.layers.BatchNormalization(),
        DefaultConv2D(filters=256),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(pool_size=2),
        
        keras.layers.Flatten(),
        keras.layers.Dense(units=128, activation='selu'),
        keras.layers.BatchNormalization(),
#         keras.layers.Dropout(0.5),
        keras.layers.Dense(units=64, activation='selu'),
        keras.layers.BatchNormalization(),
#         keras.layers.Dropout(0.5),
        keras.layers.Dense(units=10, activation='softmax'),
    ])
    
    
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])
    return model

In [ ]:
k = 10
num_val = len(X_train) // k
num_epochs = 35
all_scores = []


for i in range(k):
    print('processing fold #', i)

    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=1)
    
    # history = model.fit(X_train_part, y_train_part,
                  # epochs=num_epochs, verbose=1)
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
Epoch 1/35
1688/1688 [==============================] - 26s 8ms/step - loss: 0.3792 - accuracy: 0.8649
Epoch 2/35
1688/1688 [==============================] - 13s 8ms/step - loss: 0.2632 - accuracy: 0.9046
Epoch 3/35
1688/1688 [==============================] - 14s 8ms/step - loss: 0.2228 - accuracy: 0.9195
Epoch 4/35
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1948 - accuracy: 0.9290
Epoch 5/35
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1692 - accuracy: 0.9388
Epoch 6/35
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1451 - accuracy: 0.9460
Epoch 7/35
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1253 - accuracy: 0.9538
Epoch 8/35
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1072 - accuracy: 0.9604
Epoch 9/35
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0877 - accuracy: 0.9675
Epoch 10/35
1688/1688 [==============================

In [ ]:
all_scores

[0.934166669845581,
 0.9346666932106018,
 0.9303333163261414,
 0.9313333630561829,
 0.9353333115577698,
 0.9316666722297668,
 0.9336666464805603,
 0.9388333559036255,
 0.9301666617393494,
 0.9330000281333923]

In [ ]:
np.mean(all_scores)

0.9333166718482971

In [ ]:
def build_model():
    from functools import partial

    DefaultConv2D = partial(keras.layers.Conv2D,
                            kernel_size=2, kernel_initializer = 'he_normal',
                            activation='relu', padding="SAME")

    model = keras.models.Sequential([
        DefaultConv2D(filters=64, kernel_size=3, input_shape=[28, 28,1]),
        keras.layers.MaxPooling2D(pool_size=2),
        DefaultConv2D(filters=256),
        keras.layers.BatchNormalization(),
        DefaultConv2D(filters=256),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(pool_size=2),
        DefaultConv2D(filters=128),
        keras.layers.BatchNormalization(),
        DefaultConv2D(filters=128),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(pool_size=2),
        
        keras.layers.Flatten(),
        keras.layers.Dense(units=128, activation='selu'),
        keras.layers.BatchNormalization(),
#         keras.layers.Dropout(0.5),
        keras.layers.Dense(units=64, activation='selu'),
        keras.layers.BatchNormalization(),
#         keras.layers.Dropout(0.5),
        keras.layers.Dense(units=10, activation='softmax'),
    ])
    
    
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])
    return model

In [ ]:
k = 10
num_val = len(X_train) // k
num_epochs = 35
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=1, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
Epoch 1/35
1688/1688 [==============================] - 33s 19ms/step - loss: 0.3928 - accuracy: 0.8599
Epoch 2/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.2727 - accuracy: 0.9015
Epoch 3/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.2310 - accuracy: 0.9141
Epoch 4/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.2040 - accuracy: 0.9253
Epoch 5/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.1773 - accuracy: 0.9343
Epoch 6/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.1573 - accuracy: 0.9425
Epoch 7/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.1365 - accuracy: 0.9494
Epoch 8/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.1186 - accuracy: 0.9561
Epoch 9/35
1688/1688 [==============================] - 33s 19ms/step - loss: 0.1041 - accuracy: 0.9615
Epoch 10/35
1688/1688 [=====================

Epoch 7/35
1688/1688 [==============================] - 33s 19ms/step - loss: 0.1363 - accuracy: 0.9504
Epoch 8/35
1688/1688 [==============================] - 33s 19ms/step - loss: 0.1183 - accuracy: 0.9567
Epoch 9/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.1048 - accuracy: 0.9609
Epoch 10/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0910 - accuracy: 0.9664
Epoch 11/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0779 - accuracy: 0.9714
Epoch 12/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0689 - accuracy: 0.9748
Epoch 13/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0597 - accuracy: 0.9781
Epoch 14/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0559 - accuracy: 0.9799
Epoch 15/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0484 - accuracy: 0.9825
Epoch 16/35
1688/1688 [==============================] - 3

1688/1688 [==============================] - 33s 19ms/step - loss: 0.0584 - accuracy: 0.9786
Epoch 14/35
1688/1688 [==============================] - 33s 19ms/step - loss: 0.0546 - accuracy: 0.9806
Epoch 15/35
1688/1688 [==============================] - 33s 19ms/step - loss: 0.0491 - accuracy: 0.9820
Epoch 16/35
1688/1688 [==============================] - 33s 19ms/step - loss: 0.0416 - accuracy: 0.9849
Epoch 17/35
1688/1688 [==============================] - 33s 19ms/step - loss: 0.0425 - accuracy: 0.9844
Epoch 18/35
1688/1688 [==============================] - 33s 19ms/step - loss: 0.0380 - accuracy: 0.9862
Epoch 19/35
1688/1688 [==============================] - 33s 19ms/step - loss: 0.0330 - accuracy: 0.9883
Epoch 20/35
1688/1688 [==============================] - 33s 19ms/step - loss: 0.0327 - accuracy: 0.9883
Epoch 21/35
1688/1688 [==============================] - 33s 19ms/step - loss: 0.0279 - accuracy: 0.9898
Epoch 22/35
1688/1688 [==============================] - 32s 19ms/s

1688/1688 [==============================] - 32s 19ms/step - loss: 0.0343 - accuracy: 0.9876
Epoch 20/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0317 - accuracy: 0.9884
Epoch 21/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0299 - accuracy: 0.9891
Epoch 22/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0284 - accuracy: 0.9900
Epoch 23/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0283 - accuracy: 0.9894
Epoch 24/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0256 - accuracy: 0.9913
Epoch 25/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0229 - accuracy: 0.9920
Epoch 26/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0235 - accuracy: 0.9915
Epoch 27/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0213 - accuracy: 0.9925
Epoch 28/35
1688/1688 [==============================] - 32s 19ms/s

1688/1688 [==============================] - 32s 19ms/step - loss: 0.0249 - accuracy: 0.9912
Epoch 26/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0206 - accuracy: 0.9926
Epoch 27/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0220 - accuracy: 0.9927
Epoch 28/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0206 - accuracy: 0.9924
Epoch 29/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0204 - accuracy: 0.9930
Epoch 30/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0188 - accuracy: 0.9936
Epoch 31/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0189 - accuracy: 0.9940
Epoch 32/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0213 - accuracy: 0.9924
Epoch 33/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0166 - accuracy: 0.9942
Epoch 34/35
1688/1688 [==============================] - 32s 19ms/s

In [ ]:
all_scores

[0.9334999918937683,
 0.9244999885559082,
 0.925000011920929,
 0.9276666641235352,
 0.9350000023841858,
 0.9326666593551636,
 0.9358333349227905,
 0.9348333477973938,
 0.9316666722297668,
 0.934333324432373]

In [ ]:
np.mean(all_scores)

In [ ]:
np.mean(all_scores)

0.9314999997615814

In [ ]:
k = 10
num_val = len(X_train) // k
num_epochs = 40
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=0, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
188/188 [==============================] - 0s 1ms/step - loss: 0.3041 - accuracy: 0.8907
processing fold # 1
188/188 [==============================] - 0s 1ms/step - loss: 0.3015 - accuracy: 0.8913
processing fold # 2
188/188 [==============================] - 1s 2ms/step - loss: 0.3157 - accuracy: 0.8888
processing fold # 3
188/188 [==============================] - 0s 2ms/step - loss: 0.3156 - accuracy: 0.8897
processing fold # 4
188/188 [==============================] - 1s 2ms/step - loss: 0.3041 - accuracy: 0.8945
processing fold # 5
188/188 [==============================] - 0s 2ms/step - loss: 0.3259 - accuracy: 0.8870
processing fold # 6
188/188 [==============================] - 0s 2ms/step - loss: 0.3167 - accuracy: 0.8972
processing fold # 7
188/188 [==============================] - 0s 2ms/step - loss: 0.3355 - accuracy: 0.8867
processing fold # 8
188/188 [==============================] - 0s 2ms/step - loss: 0.3112 - accuracy: 0.8933
processing fold # 9

In [ ]:
all_scores

[0.890666663646698,
 0.8913333415985107,
 0.8888333439826965,
 0.8896666765213013,
 0.8945000171661377,
 0.8870000243186951,
 0.8971666693687439,
 0.8866666555404663,
 0.8933333158493042,
 0.890333354473114]

In [ ]:
np.mean(all_scores)

0.8909500062465667

In [ ]:
k = 10
num_val = len(X_train) // k
num_epochs = 45
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=0, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)





processing fold # 0
188/188 [==============================] - 0s 1ms/step - loss: 0.3516 - accuracy: 0.8787
processing fold # 1
188/188 [==============================] - 0s 1ms/step - loss: 0.3287 - accuracy: 0.8903
processing fold # 2
188/188 [==============================] - 0s 1ms/step - loss: 0.3544 - accuracy: 0.8808
processing fold # 3
188/188 [==============================] - 0s 1ms/step - loss: 0.3257 - accuracy: 0.8895
processing fold # 4
188/188 [==============================] - 0s 1ms/step - loss: 0.3490 - accuracy: 0.8845
processing fold # 5
188/188 [==============================] - 0s 2ms/step - loss: 0.3555 - accuracy: 0.8893
processing fold # 6
188/188 [==============================] - 0s 1ms/step - loss: 0.3313 - accuracy: 0.8872
processing fold # 7
188/188 [==============================] - 0s 1ms/step - loss: 0.3370 - accuracy: 0.8887
processing fold # 8
188/188 [==============================] - 0s 1ms/step - loss: 0.3463 - accuracy: 0.8865
processing fold # 9

In [ ]:
all_scores

[0.8786666393280029,
 0.890333354473114,
 0.8808333277702332,
 0.8895000219345093,
 0.8845000267028809,
 0.8893333077430725,
 0.8871666789054871,
 0.8886666893959045,
 0.8865000009536743,
 0.890333354473114]

In [ ]:
np.mean(all_scores)

0.8865833401679992

In [ ]:
k = 10
num_val = len(X_train) // k
num_epochs = 50
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=0, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
188/188 [==============================] - 0s 1ms/step - loss: 0.3292 - accuracy: 0.8870
processing fold # 1
188/188 [==============================] - 0s 1ms/step - loss: 0.3080 - accuracy: 0.8945
processing fold # 2
188/188 [==============================] - 0s 1ms/step - loss: 0.3321 - accuracy: 0.8878
processing fold # 3
188/188 [==============================] - 0s 1ms/step - loss: 0.3170 - accuracy: 0.8912
processing fold # 4
188/188 [==============================] - 0s 1ms/step - loss: 0.3288 - accuracy: 0.8870
processing fold # 5
188/188 [==============================] - 0s 1ms/step - loss: 0.3405 - accuracy: 0.8853
processing fold # 6
188/188 [==============================] - 0s 1ms/step - loss: 0.3224 - accuracy: 0.8975
processing fold # 7
188/188 [==============================] - 0s 1ms/step - loss: 0.3275 - accuracy: 0.8927
processing fold # 8
188/188 [==============================] - 0s 1ms/step - loss: 0.3268 - accuracy: 0.8905
processing fold # 9

In [ ]:
all_scores

[0.8870000243186951,
 0.8945000171661377,
 0.8878333568572998,
 0.8911666870117188,
 0.8870000243186951,
 0.8853333592414856,
 0.8974999785423279,
 0.8926666378974915,
 0.890500009059906,
 0.8881666660308838]

In [ ]:
np.mean(all_scores)

0.8901666760444641

In [ ]:
k = 10
num_val = len(X_train) // k
num_epochs = 55
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=0, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
188/188 [==============================] - 0s 2ms/step - loss: 0.3218 - accuracy: 0.8930
processing fold # 1
188/188 [==============================] - 0s 1ms/step - loss: 0.3283 - accuracy: 0.8868
processing fold # 2
188/188 [==============================] - 0s 1ms/step - loss: 0.3762 - accuracy: 0.8777
processing fold # 3
188/188 [==============================] - 0s 1ms/step - loss: 0.3372 - accuracy: 0.8913
processing fold # 4
188/188 [==============================] - 1s 2ms/step - loss: 0.3143 - accuracy: 0.8957
processing fold # 5
188/188 [==============================] - 0s 2ms/step - loss: 0.3195 - accuracy: 0.8995
processing fold # 6
188/188 [==============================] - 0s 2ms/step - loss: 0.3329 - accuracy: 0.8977
processing fold # 7
188/188 [==============================] - 0s 2ms/step - loss: 0.3516 - accuracy: 0.8900
processing fold # 8
188/188 [==============================] - 0s 1ms/step - loss: 0.4264 - accuracy: 0.8622
processing fold # 9

In [ ]:
all_scores

[0.8930000066757202,
 0.8868333101272583,
 0.8776666522026062,
 0.8913333415985107,
 0.8956666588783264,
 0.8995000123977661,
 0.8976666927337646,
 0.8899999856948853,
 0.8621666431427002,
 0.8636666536331177]

In [ ]:
np.mean(all_scores)

0.8857499957084656

In [ ]:
k = 10
num_val = len(X_train) // k
num_epochs = 75
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=0, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
188/188 [==============================] - 0s 1ms/step - loss: 0.3787 - accuracy: 0.8910
processing fold # 1
188/188 [==============================] - 0s 1ms/step - loss: 0.3261 - accuracy: 0.8995
processing fold # 2
188/188 [==============================] - 0s 1ms/step - loss: 0.3713 - accuracy: 0.8950
processing fold # 3
188/188 [==============================] - 0s 1ms/step - loss: 0.3727 - accuracy: 0.8918
processing fold # 4
188/188 [==============================] - 0s 1ms/step - loss: 0.3648 - accuracy: 0.8962
processing fold # 5
188/188 [==============================] - 0s 2ms/step - loss: 0.3825 - accuracy: 0.8897
processing fold # 6
188/188 [==============================] - 0s 2ms/step - loss: 0.3631 - accuracy: 0.8968
processing fold # 7
188/188 [==============================] - 0s 2ms/step - loss: 0.3685 - accuracy: 0.8942
processing fold # 8
188/188 [==============================] - 0s 2ms/step - loss: 0.4106 - accuracy: 0.8848
processing fold # 9

In [ ]:
all_scores

[0.890999972820282,
 0.8995000123977661,
 0.8949999809265137,
 0.8918333053588867,
 0.8961666822433472,
 0.8896666765213013,
 0.8968333601951599,
 0.8941666483879089,
 0.8848333358764648,
 0.8813333511352539]

In [ ]:
np.mean(all_scores)

0.8920333325862885

In [ ]:
k = 10
num_val = len(X_train) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=0, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
188/188 [==============================] - 0s 2ms/step - loss: 0.4241 - accuracy: 0.8905
processing fold # 1
188/188 [==============================] - 0s 2ms/step - loss: 0.4009 - accuracy: 0.8920
processing fold # 2
188/188 [==============================] - 0s 2ms/step - loss: 0.4051 - accuracy: 0.8967
processing fold # 3
188/188 [==============================] - 0s 2ms/step - loss: 0.4644 - accuracy: 0.8858
processing fold # 4
188/188 [==============================] - 0s 2ms/step - loss: 0.4391 - accuracy: 0.8887
processing fold # 5
188/188 [==============================] - 0s 2ms/step - loss: 0.3990 - accuracy: 0.9010
processing fold # 6
188/188 [==============================] - 0s 2ms/step - loss: 0.4117 - accuracy: 0.9022
processing fold # 7
188/188 [==============================] - 0s 2ms/step - loss: 0.4002 - accuracy: 0.9023
processing fold # 8
188/188 [==============================] - 0s 2ms/step - loss: 0.4964 - accuracy: 0.8828
processing fold # 9

In [ ]:
all_scores

[0.890500009059906,
 0.8920000195503235,
 0.8966666460037231,
 0.8858333230018616,
 0.8886666893959045,
 0.9010000228881836,
 0.9021666646003723,
 0.9023333191871643,
 0.8828333616256714,
 0.8911666870117188]

In [ ]:
np.mean(all_scores)

0.893316674232483

In [ ]:
k = 10
num_val = len(X_train) // k
num_epochs = 150
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=0, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
188/188 [==============================] - 0s 2ms/step - loss: 0.6034 - accuracy: 0.8843
processing fold # 1
188/188 [==============================] - 0s 2ms/step - loss: 0.5410 - accuracy: 0.8992
processing fold # 2
188/188 [==============================] - 0s 2ms/step - loss: 0.6295 - accuracy: 0.8812
processing fold # 3
188/188 [==============================] - 1s 2ms/step - loss: 0.5696 - accuracy: 0.8980
processing fold # 4
188/188 [==============================] - 0s 2ms/step - loss: 0.5846 - accuracy: 0.8910
processing fold # 5
188/188 [==============================] - 0s 2ms/step - loss: 0.5719 - accuracy: 0.9017
processing fold # 6
188/188 [==============================] - 1s 2ms/step - loss: 0.5268 - accuracy: 0.9028
processing fold # 7
188/188 [==============================] - 0s 2ms/step - loss: 0.5804 - accuracy: 0.8945
processing fold # 8
188/188 [==============================] - 0s 2ms/step - loss: 0.6458 - accuracy: 0.8875
processing fold # 9

In [ ]:
all_scores

[0.8843333125114441,
 0.8991666436195374,
 0.8811666369438171,
 0.8980000019073486,
 0.890999972820282,
 0.9016666412353516,
 0.9028333425521851,
 0.8945000171661377,
 0.887499988079071,
 0.890999972820282]

In [ ]:
np.mean(all_scores)

0.8931166529655457

In [ ]:
k = 10
num_val = len(X_train) // k
num_epochs = 300
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=0, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
188/188 [==============================] - 0s 2ms/step - loss: 0.7914 - accuracy: 0.8945
processing fold # 1
188/188 [==============================] - 0s 2ms/step - loss: 0.7240 - accuracy: 0.9022
processing fold # 2
188/188 [==============================] - 0s 2ms/step - loss: 0.8082 - accuracy: 0.8957
processing fold # 3
188/188 [==============================] - 0s 2ms/step - loss: 0.7843 - accuracy: 0.8977
processing fold # 4
188/188 [==============================] - 0s 2ms/step - loss: 0.7788 - accuracy: 0.8998
processing fold # 5
188/188 [==============================] - 0s 2ms/step - loss: 0.7446 - accuracy: 0.9020
processing fold # 6
188/188 [==============================] - 0s 2ms/step - loss: 0.7513 - accuracy: 0.9040
processing fold # 7
188/188 [==============================] - 0s 2ms/step - loss: 0.8099 - accuracy: 0.9012
processing fold # 8
188/188 [==============================] - 0s 2ms/step - loss: 0.8088 - accuracy: 0.8988
processing fold # 9

In [ ]:
all_scores

[0.8945000171661377,
 0.9021666646003723,
 0.8956666588783264,
 0.8976666927337646,
 0.8998333215713501,
 0.9020000100135803,
 0.9039999842643738,
 0.9011666774749756,
 0.8988333344459534,
 0.8968333601951599]

In [ ]:
np.mean(all_scores)

0.8992666721343994

In [ ]:
k = 10
num_val = len(X_train) // k
num_epochs = 37
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=0, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
188/188 [==============================] - 0s 2ms/step - loss: 0.2945 - accuracy: 0.8992
processing fold # 1
188/188 [==============================] - 0s 2ms/step - loss: 0.2995 - accuracy: 0.8933
processing fold # 2
188/188 [==============================] - 0s 2ms/step - loss: 0.3321 - accuracy: 0.8803
processing fold # 3
188/188 [==============================] - 0s 2ms/step - loss: 0.3123 - accuracy: 0.8890
processing fold # 4
188/188 [==============================] - 0s 2ms/step - loss: 0.2946 - accuracy: 0.8937
processing fold # 5
188/188 [==============================] - 0s 2ms/step - loss: 0.3071 - accuracy: 0.8957
processing fold # 6
188/188 [==============================] - 0s 2ms/step - loss: 0.3240 - accuracy: 0.8933
processing fold # 7
188/188 [==============================] - 0s 2ms/step - loss: 0.3201 - accuracy: 0.8912
processing fold # 8
188/188 [==============================] - 0s 2ms/step - loss: 0.3338 - accuracy: 0.8890
processing fold # 9

In [ ]:
all_scores

[0.8991666436195374,
 0.8933333158493042,
 0.8803333044052124,
 0.8889999985694885,
 0.893666684627533,
 0.8956666588783264,
 0.8933333158493042,
 0.8911666870117188,
 0.8889999985694885,
 0.8861666917800903]

In [ ]:
np.mean(all_scores)

0.8910833299160004

In [ ]:
k = 10
num_val = len(X_train) // k
num_epochs = 350
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=0, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0


In [ ]:
np.mean(all_scores)